In [1]:
import time
import os

import pandas as pd
import numpy as np

import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
#os.chdir('/c/Users/matth/Documents/Coding/RA Fouirnaies')

In [4]:
#!pwd

/c/Users/matth/Documents/Coding/RA Fouirnaies


# Vote Smart - FollowTheMoney Link Table

## Overview

This notebook documents the process of creating a link table between the enterprise/organization ID of Vote Smart and FollowTheMoney. We begin with a list of the desired organizations' names as seen on the Vote Smart website and scrape the FTM website search queries to obtain unique IDs. 

### Search Query Exploration

#### Example: Campaign for Working Families

Here we seek to understand the format of the search query output and create a method to extract the desired information from each query while handling/providing discussion of how to handle exceptions.

In [2]:
url = 'https://www.followthemoney.org/search-results/SearchForm?Search=Campaign+for+Working+Families'
headers = {'Access-Control-Allow-Origin': '*',
          'Access-Control-Allow-Methods': 'GET',
          'Access-Control-Allow-Headers': 'Content-Type',
          'Access-Control-Max-Age': '3600'}

Debugging for below cell

https://stackoverflow.com/questions/17597424/how-to-retrieve-the-values-of-dynamic-html-content-using-python
https://stackoverflow.com/questions/40208051/selenium-using-python-geckodriver-executable-needs-to-be-in-path

https://www.howtogeek.com/118594/how-to-edit-your-system-path-for-easy-command-line-access/amp/

In [12]:
#driver = webdriver.Firefox() #executable_path=r'your\path\geckodriver.exe'
#driver.get(url)

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path=r'C:\Users\matth\Documents\Coding\RA Fouirnaies\geckodriver.exe', options=options)
driver.get(url)


html = driver.page_source

In [14]:
start_time = time.time()

#req = requests.get(html, headers)
sample = BeautifulSoup(html, 'html.parser') #req.content

print("My program took", time.time()-start_time, "to run")

My program took 0.07480072975158691 to run


In [22]:
# navigating to dynamic search query table and obtaining a list of the 'Name' element/col of each row
query_table = sample.find_all(name='div', class_ = 'table-responsive')
query_list = query_table[0].find_all(name='td', style = 'text-align: left;')
len(query_list)

3

In [30]:
# printing unique eid for each organization
for row in query_list:
    print(row['tokenvalue'])

471
46277079
16629025


### Search & Match Automation

#### Query Creation

For both tasks we assume that we are given a list of strings where each string is an organization name that is separated by spaces (if longer than one word) and is of unknown capitalization. 

#### Organization Matching